##Importing required libraries

In [1]:
import pandas as pd
from bs4 import *
import requests
import json
import folium
import geopy
from geopy.geocoders import Nominatim

##Webscrapping data

In [2]:
#Fetching covid vaccination data
path1='https://www.mygov.in/sites/default/files/covid/vaccine/vaccine_counts_today.json'
page1 = requests.get(path1)
soup1 = BeautifulSoup(page1.content, 'html.parser')
site_json=json.loads(soup1.text)
site_json['vacc_st_data'][0]
vaccine_data=pd.DataFrame(columns=['state','dose1','dose2','total_doses'])
c=0
for i in site_json['vacc_st_data']:
  vaccine_data.loc[c]=[i['covid_state_name'],i['dose1'],i['dose2'],i['total_doses']]
  c+=1
#Fetching covid cases data
path2='https://www.mygov.in/sites/default/files/covid/covid_state_counts_ver1.json'
page2 = requests.get(path2)
soup2 = BeautifulSoup(page2.content, 'html.parser')
site_json2=json.loads(soup2.text)
active_cases=site_json2['Active']
state_names=site_json2['Name of State / UT']
total_deaths=site_json2['Death']
cured_cases=site_json2['Cured/Discharged/Migrated']
total_cases=site_json2['Total Confirmed cases']
covid_data=pd.DataFrame(columns =['state', 'total_cases','active_cases','discharged_cases','total_deaths'])
covid_data.loc[0]=[state_names['0'],total_cases['0'],active_cases['0'],cured_cases['0'],total_deaths['0']]
for i in range(0,36):
  covid_data.loc[i]=[state_names[str(i)],total_cases[str(i)],active_cases[str(i)],cured_cases[str(i)],total_deaths[str(i)]]

##Data Exploration

In [3]:
covid_data

,state,total_cases,active_cases,discharged_cases,total_deaths
0,Andaman and Nicobar,7683,5,7549,129
1,Andhra Pradesh,2073093,2149,2056501,14443
2,Arunachal Pradesh,55279,35,54964,280
3,Assam,617039,2612,608319,6108
4,Bihar,726225,32,716530,9663
5,Chandigarh,65471,66,64585,820
6,Chhattisgarh,1006833,316,992924,13593
7,Dadra and Nagar Haveli and Daman and Diu,10683,1,10678,4
8,Delhi,1440973,286,1415589,25098
9,Goa,178990,327,175279,3384


In [4]:
vaccine_data

,state,dose1,dose2,total_doses
0,Andaman and Nicobar,296964,258561,555525
1,Andhra Pradesh,34991943,25032907,60024850
2,Arunachal Pradesh,795139,612865,1408004
3,Assam,21209271,12160743,33370014
4,Bihar,54637126,27792850,82429976
5,Chandigarh,942626,635712,1578338
6,Chhattisgarh,16510938,9528852,26039790
7,Dadra and Nagar Haveli and Daman and Diu,675002,441006,1116008
8,Delhi,13927169,8929590,22856759
9,Goa,1283960,1035659,2319619


In [5]:
covid_data[['dose1','dose2','total_doses']] = vaccine_data[['dose1','dose2','total_doses']]
covid_data.head()

,state,total_cases,active_cases,discharged_cases,total_deaths,dose1,dose2,total_doses
0,Andaman and Nicobar,7683,5,7549,129,296964,258561,555525
1,Andhra Pradesh,2073093,2149,2056501,14443,34991943,25032907,60024850
2,Arunachal Pradesh,55279,35,54964,280,795139,612865,1408004
3,Assam,617039,2612,608319,6108,21209271,12160743,33370014
4,Bihar,726225,32,716530,9663,54637126,27792850,82429976


##Date Visualization

In [6]:
india=folium.Map(location = [22,78.9629],zoom_start=4.5)

In [7]:
geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")
covid_data['gcode'] = covid_data.state.apply(geolocator.geocode)
covid_data['lat'] = [g.latitude for g in covid_data.gcode]
covid_data['long'] = [g.longitude for g in covid_data.gcode]
covid_data.head()


,state,total_cases,active_cases,discharged_cases,total_deaths,dose1,dose2,total_doses,gcode,lat,long
0,Andaman and Nicobar,7683,5,7549,129,296964,258561,555525,"(Andaman, India, (10.0001051, 93.0000194))",10.000105,93.000019
1,Andhra Pradesh,2073093,2149,2056501,14443,34991943,25032907,60024850,"(Andhra Pradesh, India, (15.9240905, 80.1863809))",15.924091,80.186381
2,Arunachal Pradesh,55279,35,54964,280,795139,612865,1408004,"(Arunachal Pradesh, India, (28.0937702, 94.592...",28.093770,94.592133
3,Assam,617039,2612,608319,6108,21209271,12160743,33370014,"(Assam, India, (26.4073841, 93.2551303))",26.407384,93.255130
4,Bihar,726225,32,716530,9663,54637126,27792850,82429976,"(Bihar, India, (25.6440845, 85.906508))",25.644085,85.906508


In [8]:
for state,lat,long,total_cases,Death,Recov,Active,dose1,dose2,totaldoses in zip(list(covid_data['state']),list(covid_data['lat']),list(covid_data['long']),
                                                         list(covid_data['total_cases']),list(covid_data['total_deaths']),list(covid_data['discharged_cases']),
                                                         list(covid_data['active_cases']),list(covid_data['dose1']),list(covid_data['dose2']),
                                                         list(covid_data['total_doses'])):
    #for creating circle marker
    folium.CircleMarker(location = [lat,long],
                       radius = 5,
                       color='red',
                       fill = True,
                       fill_color="red").add_to(india)
    #for creating marker
    folium.Marker(location = [lat,long],
                  # adding information that need to be displayed on popup
                  popup=folium.Popup(('<a href="https://www.youtube.com/watch?v=aJmaw3QKMvk" target="_blank"><strong><b>State  : '+state+'</strong></a> <br>' +
                    '<strong><b>Total Cases : '+total_cases+'</strong><br>' +
                    '<strong><font color= red>Deaths : </font>'+Death+'</strong><br>' +
                    '<strong><font color=green>Recoveries : </font>'+Recov+'</strong><br>' +
                    '<strong><b>Active Cases : '+Active+'</strong><br>'+
                    '<strong><b><font color=green>Vaccine data : </font></strong> <br>' +
                    '<strong><b>Dose 1 : '+dose1+'</strong><br>' +
                    '<strong><b>Dose 2  : '+dose2+'</strong><br>' +
                    '<strong><b>Total Doses : '+totaldoses+'</strong><br>'),max_width=200)).add_to(india)
          

In [9]:
india.save(outfile= "test.html")

In [10]:
covid_data['Country']='India'

##Saving file for tableau :
https://public.tableau.com/app/profile/raghav.mundra/viz/CovidData_16356264427820/Dashboard1

In [11]:
covid_data.to_csv("./drive/My Drive/Covid_live_data.csv")

## plotly


In [12]:
covid_data.head()

,state,total_cases,active_cases,discharged_cases,total_deaths,dose1,dose2,total_doses,gcode,lat,long,Country
0,Andaman and Nicobar,7683,5,7549,129,296964,258561,555525,"(Andaman, India, (10.0001051, 93.0000194))",10.000105,93.000019,India
1,Andhra Pradesh,2073093,2149,2056501,14443,34991943,25032907,60024850,"(Andhra Pradesh, India, (15.9240905, 80.1863809))",15.924091,80.186381,India
2,Arunachal Pradesh,55279,35,54964,280,795139,612865,1408004,"(Arunachal Pradesh, India, (28.0937702, 94.592...",28.093770,94.592133,India
3,Assam,617039,2612,608319,6108,21209271,12160743,33370014,"(Assam, India, (26.4073841, 93.2551303))",26.407384,93.255130,India
4,Bihar,726225,32,716530,9663,54637126,27792850,82429976,"(Bihar, India, (25.6440845, 85.906508))",25.644085,85.906508,India


In [13]:
india